In [15]:
# !pip install PyMuPDF
# !pip install pytesseract
# !pip install "unstructured[all-docs]"

  Using cached numpy-2.2.6-cp312-cp312-macosx_10_13_x86_64.whl.metadata (62 kB)
  Using cached protobuf-5.29.5-cp38-abi3-macosx_10_9_universal2.whl.metadata (592 bytes)
  Using cached pillow-11.3.0-cp312-cp312-macosx_10_13_x86_64.whl.metadata (9.0 kB)
Using cached numpy-2.2.6-cp312-cp312-macosx_10_13_x86_64.whl (20.9 MB)
Using cached protobuf-5.29.5-cp38-abi3-macosx_10_9_universal2.whl (418 kB)
Using cached pillow-11.3.0-cp312-cp312-macosx_10_13_x86_64.whl (5.3 MB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.8
    Uninstalling protobuf-4.25.8:
      Successfully uninstalled protobuf-4.25.8
  Attempting uninstall: Pillow━━━━━━━━━━━━━━━━━━ 0/3 [protobuf]
    Found existing installation: pillow 10.4.0━━━━━━━━━━━━━━━━━━━━ 1/3 [Pillow]
    Uninstalling pillow-10.4.0:━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/3 [Pillow]
      Successfully uninstalled pillow-10.4.0━━━━━━━━━━━━━━━━━━ 1/3 [Pillow]
  Attempting uninstall: numpym╺━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/3 [Pillow]
    F

In [1]:
import os
import json
import boto3
import streamlit as st
import logging
import base64
from dotenv import load_dotenv
from requests_aws4auth import AWS4Auth
from opensearchpy import OpenSearch, RequestsHttpConnection
from opensearchpy.helpers import bulk
from unstructured.partition.pdf import partition_pdf
from io import BytesIO
from opensearchpy.exceptions import NotFoundError, ConnectionError
from streamlit_pdf_viewer import pdf_viewer
import fitz  # PyMuPDF 라이브러리
from PIL import Image # 이미지 처리를 위해 필요
import pytesseract


# 로깅 설정
logging.basicConfig(level=logging.INFO)

# ========================
# 1. 환경 변수 로드 및 클라이언트 설정
# ========================
load_dotenv()

aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
aws_region = os.getenv("AWS_REGION")
s3_bucket_name = os.getenv("S3_BUCKET_NAME")
opensearch_host = os.getenv("OPENSEARCH_COLLECTION_HOST")
opensearch_index_name = os.getenv("OPENSEARCH_KWON_NAME")
bedrock_model_id = os.getenv("BEDROCK_MODEL_ID")

if not all([aws_access_key_id, aws_secret_access_key, aws_region, s3_bucket_name, opensearch_host, opensearch_index_name, bedrock_model_id]):
    st.error("`.env` 파일에 필요한 환경 변수가 설정되지 않았습니다. 모든 변수를 채워주세요.")
    st.stop()

session = boto3.Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=aws_region
)
credentials = session.get_credentials()
awsauth = AWS4Auth(
    credentials.access_key,
    credentials.secret_key,
    aws_region,
    'aoss'
)

s3_client = session.client('s3')
bedrock_client = session.client('bedrock-runtime')
opensearch_client = OpenSearch(
    hosts=[{'host': opensearch_host, 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=30
)

# PDF 및 이미지 저장 폴더 설정 및 생성
PDF_DIR = "./multi-pdf-files"
FIGURES_DIR = "./images"
os.makedirs(PDF_DIR, exist_ok=True)
os.makedirs(FIGURES_DIR, exist_ok=True)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/kwon/miniconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/kwon/miniconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/kwon/miniconda3/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/Users/kwon

In [2]:
# ========================
# 2. 인덱싱 함수 (텍스트 + 이미지) - 다중 파일 처리
# ========================

# 현재 코드에서는 unstructured 라이브러리를 사용하고 있습니다. unstructured는 다양한 문서 형식에 범용적으로 사용되지만, PDF 파싱에 특화된 라이브러리를 사용하면 더 빠르고 정확한 결과를 얻을 수 있습니다.
# PyMuPDF (fitz): 속도와 정확성이 매우 뛰어난 라이브러리입니다. PDF의 텍스트와 레이아웃 정보를 빠르게 추출합니다.
# pdfplumber: 표(table) 추출 기능이 강력하며, 특정 영역의 텍스트나 이미지를 선택적으로 파싱하는 데 유용합니다.
# PyMuPDF는 PDF의 내부에 포함된 텍스트를 추출하는 데 매우 효과적이지만, **이미지에 포함된 텍스트(OCR)**를 직접 추출하는 기능은 기본적으로 제공하지 않습니다.
# 따라서 PyMuPDF를 사용하면서 이미지 내 텍스트를 함께 추출하려면, OCR 엔진을 별도로 연동해야 합니다.
# unstructured 라이브러리는 tesseract-ocr과 같은 OCR 엔진을 백엔드로 사용하여 이미지에서 텍스트를 추출하는 기능을 제공합니다. 현재 코드가 이미지의 텍스트를 제대로 인식하지 못하는 이유는 다음과 같을 수 있습니다.
# Tesseract OCR 엔진 미설치: unstructured 라이브러리가 의존하는 tesseract-ocr이 EC2 인스턴스에 설치되어 있지 않을 수 있습니다.
# 언어 팩 부족: Tesseract가 한글을 인식하려면 한글 언어 팩이 설치되어 있어야 합니다.
# 낮은 해상도: PDF 문서의 이미지가 너무 저해상도인 경우, OCR 성능이 저하될 수 있습니다.


def process_pdfs_and_index_opensearch():
    try:
        pdf_files = [f for f in os.listdir(PDF_DIR) if f.endswith(".pdf")]
        if not pdf_files:
            st.error(f"'{PDF_DIR}' 폴더에 PDF 파일이 없습니다.")
            return

        logging.info("OpenSearch 인덱스 삭제 및 생성 중...")
        if opensearch_client.indices.exists(index=opensearch_index_name):
            opensearch_client.indices.delete(index=opensearch_index_name)
            logging.info(f"기존 인덱스 '{opensearch_index_name}'을 성공적으로 삭제했습니다.")
        
        mapping = {
            "settings": {
                "index.knn": True,
            },
            "mappings": {
                "properties": {
                    "text": {"type": "text"},
                    "embedding": {
                        "type": "knn_vector",
                        "dimension": 1536, # <--- 이 부분을 1024에서 1536으로 변경
                        "method": {
                            "name": "hnsw",
                            "engine": "nmslib"
                        }
                    },
                    "image_paths": {"type": "keyword"},
                    "page_number": {"type": "long"},
                    "source": {"type": "keyword"}
                }
            }
        }
        opensearch_client.indices.create(index=opensearch_index_name, body=mapping)
        logging.info(f"OpenSearch 인덱스 '{opensearch_index_name}' 생성 완료.")

        docs = []
        for file_name in pdf_files:
            logging.info(f"'{file_name}' 파일 처리 중...")
            local_pdf_path = os.path.join(PDF_DIR, file_name)

            # --- 변경된 부분 시작 ---
            pages = {}
            doc = fitz.open(local_pdf_path)
            for page_num, page in enumerate(doc):
                text_content = page.get_text()
                images = []
                
                # 텍스트와 이미지 텍스트를 합칠 변수
                page_combined_text = text_content
                
                for img_index, img_info in enumerate(page.get_images(full=True)):
                    xref = img_info[0]
                    base_image = doc.extract_image(xref)
                    image_bytes = base_image["image"]
                    image_ext = base_image["ext"]
                    
                    if image_ext in ["png", "jpg", "jpeg", "bmp", "gif"]:
                        # 이미지를 BytesIO로 읽어와 PIL.Image 객체로 변환
                        img = Image.open(BytesIO(image_bytes))
                        
                        try:
                            # OCR로 이미지 내 텍스트 추출
                            ocr_text = pytesseract.image_to_string(img, lang='kor+eng')
                            page_combined_text += "\n" + ocr_text.strip()
                        except pytesseract.TesseractNotFoundError:
                            logging.error("Tesseract OCR 엔진을 찾을 수 없습니다. 시스템에 설치했는지 확인하세요.")
                        except Exception as e:
                            logging.error(f"OCR 처리 중 오류 발생: {e}")
                        
                        # 이미지를 IMAGES 폴더에 저장
                        image_path = os.path.join(FIGURES_DIR, f"{os.path.basename(file_name).split('.')[0]}_page_{page_num+1}_img_{img_index}.{image_ext}")
                        with open(image_path, "wb") as img_file:
                            img_file.write(image_bytes)
                        images.append(os.path.basename(image_path))

                pages[page_num + 1] = {'text': [page_combined_text], 'images': images}
            doc.close()

            for page_num, content in pages.items():
                if not content['text'] and not content['images']:
                    continue
                combined_text = "\n".join(content['text']).strip()

                if not combined_text:
                    logging.warning(f"'{file_name}'의 페이지 {page_num}에 텍스트가 없어 임베딩을 건너뜁니다.")
                    continue
                
                bedrock_model_id = 'amazon.titan-embed-text-v1'
                response = bedrock_client.invoke_model(
                    modelId=bedrock_model_id,
                    body=json.dumps({"inputText": combined_text})
                )
                embedding = json.loads(response['body'].read())['embedding'] # 결과 필드명 변경

                doc = {
                    'text': combined_text,
                    'source': file_name,
                    'page_number': page_num,
                    'embedding': embedding,
                    'image_paths': content['images']
                }
                docs.append({
                    '_index': opensearch_index_name,
                    '_source': doc
                })

        if docs:
            try:
                successes, failures = bulk(opensearch_client, docs)
                if failures:
                    logging.error(f"인덱싱 실패: {failures}")
                else:
                    logging.info(f"OpenSearch에 성공적으로 인덱싱되었습니다. 총 문서 수: {len(docs)}")
            except ConnectionError as e:
                logging.error(f"벌크 인덱싱 중 연결 오류: {e}")
            except Exception as e:
                logging.error(f"벌크 인덱싱 중 오류 발생: {e}")
        else:
            logging.warning("인덱싱할 문서가 없습니다.")
            
    except Exception as e:
        logging.error(f"오류 발생: {e}")

In [3]:
process_pdfs_and_index_opensearch()

INFO:root:OpenSearch 인덱스 삭제 및 생성 중...
INFO:opensearch:HEAD https://puwzk1mbr6lqzo7x97d4.us-west-2.aoss.amazonaws.com:443/test-kwon-index [status:200 request:0.618s]
INFO:opensearch:DELETE https://puwzk1mbr6lqzo7x97d4.us-west-2.aoss.amazonaws.com:443/test-kwon-index [status:200 request:0.214s]
INFO:root:기존 인덱스 'test-kwon-index'을 성공적으로 삭제했습니다.
INFO:opensearch:PUT https://puwzk1mbr6lqzo7x97d4.us-west-2.aoss.amazonaws.com:443/test-kwon-index [status:200 request:0.474s]
INFO:root:OpenSearch 인덱스 'test-kwon-index' 생성 완료.
INFO:root:'롯데온.pdf' 파일 처리 중...
INFO:root:'self.pdf' 파일 처리 중...
INFO:root:'[안내] MS Azure 교육과정 및 자격증 Guideline_200312 - 복사본.pdf' 파일 처리 중...
INFO:opensearch:POST https://puwzk1mbr6lqzo7x97d4.us-west-2.aoss.amazonaws.com:443/_bulk [status:200 request:1.281s]
INFO:root:OpenSearch에 성공적으로 인덱싱되었습니다. 총 문서 수: 60


In [10]:
# ========================
# 3. RAG 함수 (하이브리드 검색 + 이미지 출력)
# ========================

# 현재 사용 중인 cohere.embed-multilingual-v3 모델은 다국어에 강점이 있지만, 다른 Bedrock 모델을 사용해 성능을 비교해 볼 수 있습니다.
# Amazon Titan Embeddings: amazon.titan-embed-text-v1 모델은 강력한 성능과 비용 효율성을 제공합니다.

# Anthropic Claude 3 계열: claude-3-sonnet, claude-3-haiku 모델은 기존 Claude 버전에 비해 추론 능력, 속도, 비용 효율성 면에서 크게 개선되었습니다. 특히 haiku는 낮은 지연 시간과 저렴한 비용으로 빠른 답변을 제공합니다.

def get_rag_answer_from_bedrock_with_images(query):
    try:
        # 기존 코드
        # bedrock_model_id_embedding = 'cohere.embed-multilingual-v3'
        # 변경될 코드
        bedrock_model_id_embedding = 'amazon.titan-embed-text-v1'

        # response = bedrock_client.invoke_model(
        #     modelId=bedrock_model_id_embedding,
        #     body=json.dumps({"texts": [query], "input_type": "search_query"})
        # )
        # query_embedding = json.loads(response['body'].read())['embeddings'][0]

        response = bedrock_client.invoke_model(
            modelId=bedrock_model_id_embedding,
            body=json.dumps({"inputText": query})
        )
        query_embedding = json.loads(response['body'].read())['embedding']
        
        search_query = {
            "size": 3,
            "query": {
                "bool": {
                    "should": [
                        {
                            "multi_match": {
                                "query": query,
                                "fields": ["text^2"]
                            }
                        },
                        {
                            "knn": {
                                "embedding": {
                                    "vector": query_embedding,
                                    "k": 3
                                }
                            }
                        }
                    ],
                    "minimum_should_match": 1
                }
            },
            "_source": ["text", "image_paths"]
        }
        
        response = opensearch_client.search(index=opensearch_index_name, body=search_query)
        hits = response['hits']['hits']
        context_docs = []
        image_paths = []
        
        for hit in hits:
            context_docs.append(hit['_source']['text'])
            if 'image_paths' in hit['_source'] and hit['_source']['image_paths']:
                image_paths.extend(hit['_source']['image_paths'])
        
        if not context_docs:
            return "죄송합니다. 질문에 대한 관련 문서를 찾을 수 없습니다.", []

        context = "\n\n".join(context_docs)
        
        # llm_model_id = os.getenv("BEDROCK_MODEL_ID")
        # 변경 코드 (예시: Claude 3 Haiku)
        llm_model_id = 'anthropic.claude-3-haiku-20240307-v1:0'
        
        if 'claude' in llm_model_id:
            prompt = f"""
            다음은 사용자 질문에 답변하기 위한 참고 자료입니다.
            <자료>
            {context}
            </자료>
            당신은 유용한 AI 비서입니다. 제공된 자료만을 바탕으로 사용자의 질문에 한국어로 상세하고 친절하게 답변해주세요. 만약 자료에 답변이 없다면, "죄송합니다. 제공된 문서에는 답변이 없습니다."라고 말해주세요. 절대 자료에 없는 정보를 임의로 생성하지 마세요.
            사용자 질문: {query}
            답변:
            """
            body = json.dumps({"anthropic_version": "bedrock-2023-05-31", "max_tokens": 1000, "messages": [{"role": "user", "content": [{"type": "text", "text": prompt}]}], "temperature": 0.5})
            response_body = bedrock_client.invoke_model(modelId=llm_model_id, body=body, contentType="application/json", accept="application/json")
            llm_answer = json.loads(response_body.get('body').read())['content'][0]['text']
        else:
            # Titan Text 모델을 위한 프롬프트 및 호출 방식
            prompt = f"""
            You are a helpful AI assistant. Use only the provided context to answer the user's question in Korean. If the answer is not in the context, say "죄송합니다. 제공된 문서에는 답변이 없습니다." Do not make up information.
            Context:
            {context}
            Question: {query}
            Answer:
            """
            body = json.dumps({"inputText": prompt, "textGenerationConfig": {"maxTokenCount": 1000, "stopSequences": [], "temperature": 0.5, "topP": 0.9}})
            response_body = bedrock_client.invoke_model(modelId=llm_model_id, body=body)
            llm_answer = json.loads(response_body.get('body').read())['results'][0]['outputText']

        unique_image_paths = list(set(image_paths))
        return llm_answer, unique_image_paths

    except Exception as e:
        return f"RAG 처리 중 오류가 발생했습니다: {e}", []

In [14]:
# "역량 self profiling의 Skill set 단계가 무엇이 있으며 각 단계별 의미를 알려줘",
# "역량 self profiling 문의가 생기면 어디다가 연락하면 돼?",
# "역량 self profiling 도입 배경은?",
# "직무 등급 결과는 언제 오픈 되니?",
# "팀원의 Skill set 조회와 직무는 어디서 확인할 수 있나?",
# "롯데온의 판매가격 및 재고수량 설정 방법을 단계별로 알려줘.",
# "롯데온의 배송/반품정보에서 배송정보 설정 방법 알려줘.",
# "롯데온의 FAQ를 알려줘.",
# "Azure 자격증 시험 신청 방법 알려줘.",
# "Azure 자격증 획득 후 MPN 연동 방법 알려줘.",
# "Azure 자격증 신청할 때 로그인 계정은?"

## PyMuPDF 라이브러리 + OCR 추가 pytesseract(이미지 내 텍스트를 함께 추출) 사용했을 때, 롯데온은 매우 잘 됨
## unstructured 라이브러리 (tesseract-ocr과 같은 OCR 엔진을 백엔드로 사용하여 이미지에서 텍스트를 추출하는 기능을 제공) 사용 시 전반적으로 so so

query = "롯데온의 판매가격 및 재고수량 설정 방법을 단계별로 알려줘." # PDF 내용에 따라 질문을 변경하세요.

answer = get_rag_answer_from_bedrock_with_images(query)

print(f"질문: {query}")
print(f"답변: {answer}")

INFO:opensearch:POST https://puwzk1mbr6lqzo7x97d4.us-west-2.aoss.amazonaws.com:443/test-kwon-index/_search [status:200 request:0.488s]


질문: 롯데온의 판매가격 및 재고수량 설정 방법을 단계별로 알려줘.
답변: ("롯데온의 판매가격 및 재고수량 설정 방법은 다음과 같습니다.\n\n1. 모든 옵션의 판매가격 및 재고수량을 동일하게 입력하는 경우:\n   - 상단에 판매가격과 재고수량을 입력합니다.\n   - 각각 '선택목록 일괄수정'을 클릭하면 모든 옵션의 판매가격과 재고수량이 동일하게 설정됩니다.\n\n2. 옵션별로 다른 값을 입력하는 경우:\n   - 하단의 스크롤을 오른쪽으로 드래그하여 각 옵션의 판매가격, 재고수량, 상품총용량 값을 각각 입력합니다.\n   - 예를 들어, 300ml, 20개 포장 생수라면 상품총용량은 300 x 20 = 6000으로 입력해야 합니다.\n   - 상품총용량 등록 시 기준용량에 따른 단위별 가격이 자동으로 계산되어 노출됩니다.\n\n이와 같은 방법으로 롯데온에서 판매가격 및 재고수량을 설정할 수 있습니다.", ['롯데온_page_16_img_2.jpeg', '롯데온_page_16_img_4.jpeg', '롯데온_page_33_img_0.jpeg', '롯데온_page_16_img_6.jpeg', '롯데온_page_16_img_3.jpeg', '롯데온_page_18_img_0.jpeg', '롯데온_page_16_img_1.jpeg', '롯데온_page_16_img_0.jpeg', '롯데온_page_18_img_1.jpeg', '롯데온_page_33_img_1.jpeg', '롯데온_page_33_img_3.jpeg', '롯데온_page_16_img_5.jpeg', '롯데온_page_33_img_2.jpeg', '롯데온_page_16_img_7.jpeg'])
